In [ ]:
# importing the sys module
import sys
sys.path.append('C:/Users/nsshi/github/Coeus/src/modules')  
import lib
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
import numpy as np
import tensorflow as tf

# File data
save_dir = '/modelsOut/'
emg_filename = r'Data\EMG_Mar_19_09_34_15.txt'
q_filename = r'Data\Q_Mar_19_09_33_45.txt'

# Preprocessing
emg, emgTimes, q, qTimes = lib.import_files(emg_filename, q_filename, old=True)
qF, emgF, tF, timestampsF = lib.unify_timeseries_high(emg, emgTimes, q, qTimes)
emgModel = lib.normalize_emg_rolling(emgF)
qModel = lib.finger_nodes(qF, finger = 'indextip')

In [ ]:
sequence_length = 100
trn, val, tst, input_shape, output_shape = lib.create_dataset(tF, emgModel, qModel, sequence_length = sequence_length)

In [ ]:
# Dropout
do=0.8

inputs = keras.Input(shape=(sequence_length, input_shape))
x = layers.Dense(12, activation="relu")(inputs)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)

# Recurrent layers
x = layers.GRU(4, recurrent_dropout=do, return_sequences=True)(x)
x = layers.GRU(4, recurrent_dropout=do, return_sequences=True)(x)
x = layers.GRU(4, recurrent_dropout=do)(x)
x = layers.Dropout(do)(x)

# Dense layers
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(16, activation="relu")(x)
x = layers.Dense(8, activation="relu")(x)
x = layers.Dense(4, activation="relu")(x)

# Output layer
outputs = layers.Dense(output_shape)(x)

# Construct model and descent algorithm, train, and print test results
model = keras.Model(inputs, outputs)
callbacks = [
 keras.callbacks.ModelCheckpoint(save_dir + "Final_Model.keras",
 save_best_only=True)
]

opt = tf.keras.optimizers.Adam(learning_rate=0.005,
    beta_1=0.99,
    beta_2=0.999,
    epsilon=1e-05,
)
model.compile(optimizer=opt, loss="mse", metrics=[tf.keras.metrics.RootMeanSquaredError()])
history = model.fit(trn,
 epochs=40,
 validation_data=val,
 callbacks=callbacks)
model = keras.models.load_model(save_dir + "Final_Model".keras")
print(f"Test RMSE: {model.evaluate(tst)[1]:.4f}")